# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.633325044898082
 1.3572234269930643
 1.0136748932191715

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.874727   1.0       0.0
 0.16238   -0.878923  1.0
U factor:
3×3 Matrix{Float64}:
 0.576937   0.711172  0.345216
 0.0       -0.48285   0.0678111
 0.0        0.0       0.727217

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.874727   1.0       0.0
 0.16238   -0.878923  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.576937   0.711172  0.345216
 0.0       -0.48285   0.0678111
 0.0        0.0       0.727217

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999999
 1.0

In [8]:
Alu\b

3-element Vector{Float64}:
 1.0000000000000002
 0.9999999999999999
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.747119  -0.336615  -0.573152
 -0.121317  -0.778744   0.6155
 -0.653525   0.529385   0.540977
R factor:
3×3 Matrix{Float64}:
 -0.772215  -0.687817  -0.587373
  0.0       -0.586103  -0.484004
  0.0        0.0        0.447602

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.747119  -0.336615  -0.573152
 -0.121317  -0.778744   0.6155
 -0.653525   0.529385   0.540977

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -0.772215  -0.687817  -0.587373
  0.0       -0.586103  -0.484004
  0.0        0.0        0.447602

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.01931232936700001
  0.31392814844897343
  2.678555380835414
vectors:
3×3 Matrix{Float64}:
 -0.315403   0.728896   -0.607645
 -0.43229   -0.68039    -0.591773
  0.844776  -0.0760316  -0.529691

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -0.01931232936700001
  0.31392814844897343
  2.678555380835414

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.315403   0.728896   -0.607645
 -0.43229   -0.68039    -0.591773
  0.844776  -0.0760316  -0.529691

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0          -8.88178e-15  -1.77636e-15
 -7.10543e-15   1.0           1.77636e-15
  1.06581e-14   1.77636e-14   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-2.1450408521852893

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.464602 seconds (134.33 k allocations: 15.463 MiB, 45.61% compilation time)


In [23]:
@time eigvals(Asym_noisy);

  1.507659 seconds (13 allocations: 7.920 MiB, 1.25% gc time)


In [24]:
@time eigvals(Asym_explicit);

  0.334406 seconds (5.93 k allocations: 8.357 MiB, 5.10% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.241577 seconds (851.66 k allocations: 230.886 MiB, 17.63% gc time, 35.62% compilation time)


6.394132873055858

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 9//10  1//1   7//10
 1//2   4//5   1//10
 7//10  3//10  4//5

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 13//5
  7//5
  9//5

In [29]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1    0//1   0//1
 7//9    1//1   0//1
 5//9  -22//43  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 9//10    1//1     7//10
 0//1   -43//90   23//90
 0//1     0//1   -34//215

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [ ]:
using LinearAlgebra

In [39]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

A_eigv = eigen(A)
A_eigv = [A_eigv.values]

1-element Vector{Vector{Float64}}:
 [-128.49322764802145, -55.887784553057, 42.752167279318854, 87.16111477514494, 542.4677301466137]

In [40]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

LoadError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [44]:
A_eigv = eigen(A)
Diagonal(A_eigv.vectors)

5×5 Diagonal{Float64, Vector{Float64}}:
 -0.147575   ⋅         ⋅         ⋅           ⋅ 
   ⋅        0.173068   ⋅         ⋅           ⋅ 
   ⋅         ⋅        0.422161   ⋅           ⋅ 
   ⋅         ⋅         ⋅        0.0366447    ⋅ 
   ⋅         ⋅         ⋅         ⋅         -0.364928

In [ ]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [ ]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback

Please click on `Validate` on the top, once you are done with the exercises.